<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguage Natural

## Nota:
`Esta práctica la realizamos en conjunto con Norma Yuliana Cala` https://github.com/normacalamartinez

## Taller #6: Word2Vec
`Fecha de entrega: Septiembre 19, 2020. (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`



## Libre elección

Expectativas:
- Uso de Word2Vec 
- Responder:
    - ¿Su modelo da buenos resultados? ¿Por qué sí o por qué no?
    - ¿Qué problemas encontró al realizar este taller?
    
    
Ideas algunas por si están varados: 
- Hacer más visualizaciones
- Jugar con los parámetros de la funcion word2vec de gensim
- Usar alguna función de gensim que no hayamos visto en clase (https://radimrehurek.com/gensim/models/word2vec.html#module-gensim.models.word2vec)
- Comparar palabras del inglés y el español
- Encontrar analogías y similutdes interesantes


### Enviar código con explicaciones del por qué escogieron ese camino

-------------------------------------------

### Sobre los datos

Los datos provienen de dialogos en español de una serie llamada `Los soprano`. Link:  https://www.kaggle.com/alexol/spanish-movie-dialogues 
Contiene 86 archivos de dialogos.

---------------

#### Librerias

In [25]:
import pandas as pd
import os
import re

from nltk.corpus import stopwords
stopwords_sp = stopwords.words('spanish')

#### Funciones

In [ ]:
def pre_procesado(texto):
    """ pasa a minuscula, remueve caracteres especiales, numeros, palabras vacias a cada palabra"""
    texto = texto.lower()
    texto = re.sub(r"[\W\d_]+", " ", texto)
    texto = " ".join([palabra for palabra in texto.split() if palabra not in stopwords_sp])
    return texto

In [31]:
def get_file_names_in_path(path):
    """ Devuelve una lista con los nombres de archivos en la ruta [path] """
    filenames = []
    try:
        for raiz, dirs, archivos in os.walk(path):
            for archivo in archivos:
                filenames.append(path+archivo)
        return filenames
    except Exception as e:
        print('Error leyendo los archivo en [{}] Mensaje: {}'.format(path, str(e)))
    return None

In [32]:
def read_file_data(file_list):
    documentos = []
    for file in file_list:
        with open(file) as archivo: 
            data = archivo.read()
            data = pre_procesado(data)
            documentos.append(data.split())
    return documentos

### Proceso

In [33]:
%%time
# Lectura de archivos con dialogos de la serie.
filanames = get_file_names_in_path('archivos/')
filanames

# Obtención de los documentos
documentos = read_file_data(filanames)

CPU times: user 947 ms, sys: 17.1 ms, total: 964 ms
Wall time: 991 ms


In [34]:
print(f"Hay {len(documentos)} documentos")

Hay 86 documentos


### Lista de lista de palabras

In [35]:
documentos

[['proshai',
  'livushka',
  'pinza',
  'maldita',
  'pinza',
  'nadie',
  'deja',
  'lugar',
  'dios',
  'tony',
  'dios',
  'tony',
  'bien',
  'tío',
  'ben',
  'ocurrido',
  'hola',
  'haces',
  'aquí',
  'venir',
  'imbécil',
  'robó',
  'video',
  'reproductor',
  'sala',
  'pública',
  'enemigo',
  'público',
  'película',
  'genial',
  'clase',
  'terminó',
  'vamos',
  'mil',
  'año',
  'mires',
  'películas',
  'viejas',
  'quién',
  'compañero',
  'clase',
  'bien',
  'madre',
  'gusta',
  'lavanda',
  'lista',
  'segundo',
  'señor',
  'soprano',
  'hola',
  'noah',
  'tannembaum',
  'cómo',
  'agradezco',
  'dejara',
  'ver',
  'película',
  'aquí',
  'altavoces',
  'bose',
  'marcan',
  'diferencia',
  'gusta',
  'cine',
  'gente',
  'dice',
  'hawks',
  'inventó',
  'género',
  'scarface',
  'cagney',
  'modernidad',
  'muni',
  'hizo',
  'así',
  'doy',
  'voto',
  'william',
  'wellman',
  'noah',
  'voy',
  'buscar',
  'cd',
  'barenaked',
  'ladies',
  'vamos',
  'úl

### Uso de Word2Vec

In [ ]:
! pip install gensim
import gensim.models.word2vec as w2v

### Inicialización del modelo

In [38]:
%%time

mi_modelo = w2v.Word2Vec(documentos,
                            sg=1, # 1 para usar skip-gram, 0 para usar CBOW
                            seed=1, # 1 para tener resultados reproducibles y debugear
                            size=256, # el tamaño de los vectores de palabras, o neuronas
                            min_count=50, #mínimo de veces que ha de aparecer cada palabra, para ser considerada,
                            window=12) # ventana contextual de cada palabra

CPU times: user 6.29 s, sys: 49.4 ms, total: 6.34 s
Wall time: 2.5 s


#### Busquemos palabras similares

In [40]:
mi_modelo.wv.most_similar("familia")[:3]

[('respeto', 0.8805289268493652),
 ('amigos', 0.8536269664764404),
 ('muerte', 0.8354512453079224)]

In [57]:
mi_modelo.wv.most_similar("hola")[:3]

[('verte', 0.8643296957015991),
 ('ro', 0.850563645362854),
 ('traje', 0.8361308574676514)]

#### Probemos con analogias

In [48]:
def nearest_similarity_cosmul(start1, end1, end2):
    similarities = mi_modelo.wv.most_similar_cosmul(
        positive=[end2, start1],
        negative=[end1]
    )
    start2 = similarities[0][0]
    print("{0} es a {1}, lo que {2} es a {3}".format(start1, end1, start2, end2))

In [79]:
nearest_similarity_cosmul("abuela", "mujer", "padre")

abuela es a mujer, lo que papá es a padre


#### ¿Qué tan similares son dos palabras?

In [58]:
similarity = mi_modelo.similarity("hombre","mujer")
similarity

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.


0.5699707

#### Busca palabras similares dada una palabra

In [60]:
resultado = mi_modelo.similar_by_word("familia")
resultado

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similar_by_word` (Method will be removed in 4.0.0, use self.wv.similar_by_word() instead).
  """Entry point for launching an IPython kernel.


[('respeto', 0.8805289268493652),
 ('amigos', 0.8536269664764404),
 ('muerte', 0.8354512453079224),
 ('único', 0.8353354930877686),
 ('hablo', 0.834059476852417),
 ('asunto', 0.832849383354187),
 ('tema', 0.8244525194168091),
 ('ello', 0.8223918676376343),
 ('decía', 0.8196921348571777),
 ('respecto', 0.8171603679656982)]

#### Similitud entre frases

In [71]:
resultado = mi_modelo.n_similarity(["hola", "padre", "bien"], ["bueno", "madre", "vemos"])
resultado

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).
  """Entry point for launching an IPython kernel.


0.82014894

#### Consultemos la entidad más parecida entre una lista de entidades

In [86]:
resultado = mi_modelo.wv.most_similar_to_given("padre",["hombre", "niño", "bebé"])
resultado

'hombre'

#### ¿Qué tan distantes están las entidades dadas?

In [88]:
resultado = mi_modelo.wv.rank("padre","hombre")
resultado

4

In [89]:
resultado = mi_modelo.wv.rank("padre","bebé")
resultado

556

In [91]:
resultado = mi_modelo.wv.rank("padre","niño")
resultado

64

In [93]:
resultado = mi_modelo.wv.rank("padre","madre")
resultado

174

#### 🤔 ¿Da igual si se hace el `rank` invertido?

Pensabamos que si debía dar igual, pero leyendo la documentación resulta que compara distancias con las entidades a su alrededor. Naturalmente, las entidades alrededor son diferentes al menos en algunos lados.

In [95]:
resultado = mi_modelo.wv.rank("madre","padre") == mi_modelo.wv.rank("padre","madre")
resultado

False

In [97]:
resultado1 = mi_modelo.wv.rank("madre","padre")
resultado2 = mi_modelo.wv.rank("padre","madre")
print(resultado1)
print(resultado2) 

147
174


### ¿Su modelo da buenos resultados? ¿Por qué sí o por qué no?

No. El vocabulario es pobre y tiene pocas relaciones con entidades que deberia tener. Por ejemplo: madre-mujer, familia-hogar, bueno-bien etc. Tiene muchas palabras pero no tiene tantos sinonimos y no nos permitió hacer buenas analogías.

### ¿Qué problemas encontró al realizar este taller?

Cuando estabamos implementando el modelo ya al usarlo, no encontrabamos palabras, empezamos a ingresar palabras al azar y salia errores que decia que la palabra no está en el vocabulario.

Tecnicamente hablando, no encontramos problemas.

Al principio si fue dificil encontrar los datos. De hecho, sigue siendo dificil encontrar buenos datos, completos y de calidad y en Español.